# Parallel Search

## Implementation with 'threading'

In [1]:
import threading

In [2]:
class SearchStrategy(object):
    def __init__(self):
        self._threads = []
        self._arrays_remaining = 0
        self._success_event = threading.Event()
        self._lock = threading.Lock()

    def __call__(self, target_item, arrays):
        # intiialise threads
        self._arrays_remaining = len(arrays)
        for array in arrays:
            t = threading.Thread(target=self._search, args=(target_item, array))
            self._threads.append(t)
        # begin threads
        for t in self._threads:
            t.start()
        # await result
        while True:
            if self._success_event.is_set():
                return True
            elif self._arrays_remaining == 0:
                return False
            else:
                continue
    
    def _search(self, target_item, array):
        """Search for target_item in array, and set success_event when found."""
        print(f"Searching for {target_item} in {array}\n")
        # manually search to enable quick exit
        for item in array:
            if item == target_item:
                print(f"Found {target_item} in {array}\n")
                self._success_event.set()
            if self._success_event.is_set(): # quick exit
                print(f"Search of {array} is terminated\n")
                break
        else:
            print(f"Exhausted {array}\n")
        with self._lock: # race condition possible: two threads may load and set same values
            self._arrays_remaining -= 1 # (resolves to four opcodes)

In [13]:
arrays = [[1,2,3], [4,5,6], [7,8,9], [10,11,12]]
target_item = 8
search_strategy = SearchStrategy()
result = search_strategy(target_item, arrays)

Searching for 8 in [1, 2, 3]
Searching for 8 in [4, 5, 6]

Exhausted [1, 2, 3]


Exhausted [4, 5, 6]

Searching for 8 in [7, 8, 9]

Found 8 in [7, 8, 9]

Search of [7, 8, 9] is terminated

Searching for 8 in [10, 11, 12]

Search of [10, 11, 12] is terminated

